### Imports

In [17]:
from tqdm import tqdm
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_mtsim
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/")
from gym_mtsim_forked.gym_mtsim.data import FOREX_DATA_PATH_TRAIN, FOREX_DATA_PATH_TEST, FOREX_DATA_PATH
from gym_mtsim import OrderType, Timeframe, MtEnv, MtSimulator
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, STATUS_FAIL
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
import time
import torch

# Parameters

In [18]:
# import pickle
# import time
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


# def objective(x):
#     return {
#         'loss': x ** 2,
#         'status': STATUS_OK,
#         # -- store other results like this
#         'eval_time': time.time(),
#         'other_stuff': {'type': None, 'value': x},
#         # -- attachments are handled differently
#         'attachments':
#             {'time_module': pickle.dumps(time.time)}
#         }
# trials = Trials()
# best = fmin(objective,
#             space=hp.uniform('x', -10, 10),
#             algo=tpe.suggest,
#             max_evals=100,
#             trials=trials)

# print(best)

In [19]:
# trials.results

In [20]:
import pickle
# unpack the pickle file and load the data that is in symbols_forex.pkl
with open('C:/Users/WilliamFetzner/Documents/Trading/gym_mtsim_forked/gym_mtsim/data/symbols_forex.pkl', 'rb') as f:
    symbols = pickle.load(f)
# convert symbols to a pd.dataframe
# symbols[1]['EURUSD']
split = int(len(symbols[1]['EURUSD']) * 0.80)
validation_split = int(len(symbols[1]['EURUSD']) * 0.90)

In [21]:
# get the 2 weeks of the symbols[1]['EURUSD'] dataframe by first finding the max date
# then subtracting 14 days from that date
symbols[1]['EURUSD'].index = pd.to_datetime(symbols[1]['EURUSD'].index)
max_date = symbols[1]['EURUSD'].index.max()
two_weeks = max_date - pd.DateOffset(days=14)
one_week = max_date - pd.DateOffset(days=7)

In [22]:
training_index_slice = symbols[1]['EURUSD'].loc[:two_weeks, :].index
validation_index_slice = symbols[1]['EURUSD'].loc[two_weeks:one_week, :].index
testing_index_slice = symbols[1]['EURUSD'].loc[one_week:, :].index

In [23]:
validation_index_slice

DatetimeIndex(['2024-04-03 00:00:00+00:00', '2024-04-03 01:00:00+00:00',
               '2024-04-03 02:00:00+00:00', '2024-04-03 03:00:00+00:00',
               '2024-04-03 04:00:00+00:00', '2024-04-03 05:00:00+00:00',
               '2024-04-03 06:00:00+00:00', '2024-04-03 07:00:00+00:00',
               '2024-04-03 08:00:00+00:00', '2024-04-03 09:00:00+00:00',
               ...
               '2024-04-09 15:00:00+00:00', '2024-04-09 16:00:00+00:00',
               '2024-04-09 17:00:00+00:00', '2024-04-09 18:00:00+00:00',
               '2024-04-09 19:00:00+00:00', '2024-04-09 20:00:00+00:00',
               '2024-04-09 21:00:00+00:00', '2024-04-09 22:00:00+00:00',
               '2024-04-09 23:00:00+00:00', '2024-04-10 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Time', length=121, freq=None)

### Create Env

In [24]:
sim_train = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_train = MtEnv(
    original_simulator=sim_train,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(training_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

In [25]:
sim_validation = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_validation = MtEnv(
    original_simulator=sim_validation,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(validation_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

In [26]:
sim_testing = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_testing = MtEnv(
    original_simulator=sim_testing,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(testing_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

### Define Functions

In [27]:
def print_stats(reward_over_episodes, printing_name):
    """  Print Reward  """

    avg_rewards = np.mean(reward_over_episodes)
    min_rewards = np.min(reward_over_episodes)
    max_rewards = np.max(reward_over_episodes)

    print (f'Min. {printing_name}          : {min_rewards:>10.3f}')
    print (f'Avg. {printing_name}          : {avg_rewards:>10.3f}')
    print (f'Max. {printing_name}          : {max_rewards:>10.3f}')

    return min_rewards, avg_rewards, max_rewards


# ProgressBarCallback for model.learn()
class ProgressBarCallback(BaseCallback):

    def __init__(self, check_freq: int, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        self.progress_bar = tqdm(total=self.model._total_timesteps, desc="model.learn()")

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            self.progress_bar.update(self.check_freq)
        return True
    
    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        self.progress_bar.close()


In [28]:
space = {
    'learning_rate': hp.loguniform('learning_rate', -5, -2), # Learning rate
    # 'gamma': hp.uniform('gamma', 0.97, 0.99), # Discount factor
    'ent_coef': hp.loguniform('ent_coef', -5, 0), # Entropy coefficient
    # 'learning_timesteps': hp.choice('learning_timesteps', [25, 50, 100, 250, 500]),
    'window_size': hp.choice('window_size', [10, 20, 50]) # Window size
}

In [29]:
# TRAINING + TEST
def train_val_model(model, model_policy, env_tr, env_val, seed, steps_str, window_size_param, lr, gamma_param, entropy, total_learning_timesteps=10_000):
    """
    Trains and validates a model using the Proximal Policy Optimization (PPO) algorithm.

    Args:
        model (object): The model to be trained.
        model_policy (object): The policy used by the model.
        env_tr (object): The training environment.
        env_val (object): The validation environment.
        seed (int): The random seed for reproducibility.
        steps_str (str): A string representing the number of steps.
        window_size_param (int): The window size parameter.
        lr (float): The learning rate.
        gamma_param (float): The gamma parameter.
        entropy (float): The entropy coefficient.
        total_learning_timesteps (int, optional): The total number of learning timesteps. Defaults to 10,000.

    Returns:
        tuple: A tuple containing the reward over validations, orders over validations, and the model dictionary.
    """
    # reproduce training and test
    print('-' * 80)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    #model_dict = {}
    env_tr.window_size = window_size_param
    print(f'Window size: {window_size_param}, entropy: {entropy}, learning rate: {lr}')
    # eval_callback = EvalCallback(env_tr, log_path='./logs/', eval_freq=1000)
    model = PPO(model_policy, env_tr, verbose=0, ent_coef=entropy, learning_rate=lr)#, gamma=gamma_param, 
    obs_tr, info_tr = env_tr.reset(seed=seed)
    # custom callback for 'progress_bar'
    model.learn(total_timesteps=total_learning_timesteps)#, callback=ProgressBarCallback(100))

    reward_over_validations = []
    orders_over_validations = []

    tbar = tqdm(range(0, 10))

    for episode in tbar:
        env_val.window_size = window_size_param
        obs_val, info_val = env_val.reset(seed=seed)

        total_reward = 0
        done_val = False

        while not done_val:
            action, _states = model.predict(obs_val)
            obs_val, reward_val, terminated_val, truncated_val, info_val = env_val.step(action)
            done_val = terminated_val or truncated_val

            total_reward += reward_val
            if done_val:
                break
        try:
            order_len = len(env_val.render()['orders'])
        except:
            order_len = 0

        # model_dict[f'model_{episode}'] = model
        # model.save(f'models_4_19_24/window_{window_size_param}_entropy_{round(entropy, 4)}/model_{steps_str}_{episode}.pkl')

        reward_over_validations.append(total_reward)    
        orders_over_validations.append(order_len)  


        # if episode % 1 == 0:
        avg_reward = np.mean(reward_over_validations)
        avg_orders = np.mean(orders_over_validations)
        tbar.set_description(f'Episode: {episode}, Avg. Reward: {avg_reward:.3f}, # of orders: {avg_orders:.3f}')
        tbar.update()

    tbar.close()
    avg_reward = np.mean(reward_over_validations)

    return reward_over_validations, orders_over_validations#, model_dict

### Train + Test Env

In [30]:
seed = 2024  # random seed
total_num_episodes = 10

# print ("env_name                 :", env_name)
print ("seed                     :", seed)

# INIT matplotlib
plot_settings = {}
plot_data = {'x': [i for i in range(1, total_num_episodes + 1)]}

# learning_timesteps_list_in_K = [25]#, 50, 100]
# learning_timesteps_list_in_K = [50, 250, 500]
# learning_timesteps_list_in_K = [500, 1000, 3000, 5000]

# RL Algorithms: https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

timesteps_models_dict = {}
def objective(params):
    window_size = params['window_size']
    learning_timesteps = 50 #params['learning_timesteps']
    ent_coef = params['ent_coef']
    gamma = 0.99 #params['gamma']
    learning_rate = params['learning_rate']#0.0003#

    total_learning_timesteps = learning_timesteps * 1000
    step_key = f'{learning_timesteps}K'
    policy_dict = PPO.policy_aliases
    policy = policy_dict.get('MultiInputPolicy')
    class_name = type(PPO).__qualname__
    plot_key = f'{class_name}_rewards_'+step_key
    try:
        rewards, orders = train_val_model(PPO, policy, env_train, env_validation, seed, step_key, window_size, 
                                                    learning_rate, gamma, ent_coef, total_learning_timesteps)
    except:
        print(f'''there was an error with those parameters: Window: {window_size}, timesteps: {learning_timesteps}, \n
              ent_coef: {ent_coef}, gamma: {gamma}, learning_rate: {learning_rate}''')
        return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}
    # timesteps_models_dict[step_key] = models_dict
    min_rewards, avg_rewards, max_rewards, = print_stats(rewards, 'Reward')
    print_stats(orders, 'Orders')
    label = f'Avg. {avg_rewards:>7.2f} : {class_name} - {step_key}'
    plot_data[plot_key] = rewards
    plot_settings[plot_key] = {'label': label}     

    return {'loss': -avg_rewards, 'status': STATUS_OK, 'eval_time': time.time(), 'parameters': params} 

seed                     : 2024


In [31]:
# check if it is working:
parameters = {
    'window_size': 10,
    'learning_timesteps': 25,
    'ent_coef': 0.024532025239255142,
    'gamma': 0.9484679718228304,
    'learning_rate': 0.04654130844370433
}
objective(parameters)

--------------------------------------------------------------------------------
Window size: 10, entropy: 0.024532025239255142, learning rate: 0.04654130844370433


Episode: 9, Avg. Reward: 6194.195, # of orders: 40.100: 100%|██████████| 10/10 [00:23<00:00,  2.30s/it] 

Min. Reward          : -109204.587
Avg. Reward          :   6194.195
Max. Reward          : 165644.167
Min. Orders          :     31.000
Avg. Orders          :     40.100
Max. Orders          :     52.000


{'loss': -6194.194722510193,
 'status': 'ok',
 'eval_time': 1713569432.4935591,
 'parameters': {'window_size': 10,
  'learning_timesteps': 25,
  'ent_coef': 0.024532025239255142,
  'gamma': 0.9484679718228304,
  'learning_rate': 0.04654130844370433}}

In [32]:
# PPO('MultiInputPolicy', env_train, verbose=0, ent_coef=parameters['ent_coef']).learn(total_timesteps=25_000) #, learning_rate=parameters['learning_rate'], gamma=parameters['gamma'], ent_coef=parameters['ent_coef']

In [33]:
trials = Trials()
best = fmin(fn=objective,
             space=space,
             algo=tpe.suggest,
             max_evals=250, # Number of evaluations of the objective function
             trials=trials,
             trials_save_file='hyperopt/trials_04_19.pkl')

print("Best parameters:", best)

--------------------------------------------------------------------------------
Window size: 50, entropy: 0.09564948621047711, learning rate: 0.09777522274138992
there was an error with those parameters: Window: 50, timesteps: 50, 

              ent_coef: 0.09564948621047711, gamma: 0.99, learning_rate: 0.09777522274138992
--------------------------------------------------------------------------------
Window size: 10, entropy: 0.023492586731907246, learning rate: 0.030788308384088782
  0%|          | 1/250 [04:02<16:46:19, 242.49s/trial, best loss=?]

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: -28921.789, # of orders: 41.000:   0%|          | 0/10 [00:02<?, ?it/s]
Episode: 0, Avg. Reward: -28921.789, # of orders: 41.000:  10%|#         | 1/10 [00:02<00:21,  2.40s/it]
Episode: 1, Avg. Reward: -34085.605, # of orders: 43.000:  20%|##        | 2/10 [00:04<00:19,  2.40s/it]
Episode: 1, Avg. Reward: -34085.605, # of orders: 43.000:  30%|###       | 3/10 [00:04<00:10,  1.49s/it]
Episode: 2, Avg. Reward: -42739.601, # of orders: 45.667:  40%|####      | 4/10 [00:07<00:08,  1.49s/it]
Episode: 2, Avg. Reward: -42739.601, # of orders: 45.667:  50%|#####     | 5/10 [00:07<00:06,  1.32s/it]
Episode: 3, Avg. Reward: -57238.624, # of orders: 43.750:  50%|#####     | 5/10 [00:09<00:06,  1.32s/it]
Episode: 3, Avg. Reward: -57238.624, # of orders: 43.750:  60%|######    | 6/10 [00:09<00:06,  1.65s/it]
Episode: 4, Avg. Reward: -56452.200, # of orders: 43.600:  70%|#######   | 7/10 [00:11<00:04,  1.65s/it]
Episode: 4, Avg. Reward:

Min. Reward          : -100735.695                                 
Avg. Reward          : -45402.295                                  
Max. Reward          :  18283.828                                  
Min. Orders          :     38.000                                  
Avg. Orders          :     44.800                                  
Max. Orders          :     51.000                                  
--------------------------------------------------------------------------------    
Window size: 20, entropy: 0.3525967673948514, learning rate: 0.022798376214278465   
there was an error with those parameters: Window: 20, timesteps: 50,                

              ent_coef: 0.3525967673948514, gamma: 0.99, learning_rate: 0.022798376214278465
--------------------------------------------------------------------------------    
Window size: 50, entropy: 0.11126832887943819, learning rate: 0.017296762174125433  
there was an error with those parameters: Window: 50, timesteps: 50,     

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: 50633.164, # of orders: 10.000:   0%|          | 0/10 [00:02<?, ?it/s]
Episode: 0, Avg. Reward: 50633.164, # of orders: 10.000:  10%|#         | 1/10 [00:02<00:21,  2.34s/it]
Episode: 1, Avg. Reward: 25470.367, # of orders: 10.000:  20%|##        | 2/10 [00:04<00:18,  2.34s/it]
Episode: 1, Avg. Reward: 25470.367, # of orders: 10.000:  30%|###       | 3/10 [00:04<00:09,  1.42s/it]
Episode: 2, Avg. Reward: 18910.785, # of orders: 10.333:  40%|####      | 4/10 [00:06<00:08,  1.42s/it]
Episode: 2, Avg. Reward: 18910.785, # of orders: 10.333:  50%|#####     | 5/10 [00:06<00:06,  1.28s/it]
Episode: 3, Avg. Reward: 701.158, # of orders: 11.000:  50%|#####     | 5/10 [00:09<00:06,  1.28s/it]  
Episode: 3, Avg. Reward: 701.158, # of orders: 11.000:  60%|######    | 6/10 [00:09<00:06,  1.52s/it]
Episode: 4, Avg. Reward: 10622.697, # of orders: 11.400:  70%|#######   | 7/10 [00:11<00:04,  1.52s/it]
Episode: 4, Avg. Reward: 10622.697,

Min. Reward          : -91468.290                                                   
Avg. Reward          :  -2706.836                                                   
Max. Reward          :  61616.269                                                   
Min. Orders          :      6.000                                                   
Avg. Orders          :     11.800                                                   
Max. Orders          :     20.000                                                   
--------------------------------------------------------------------------------     
Window size: 20, entropy: 0.01405446007531199, learning rate: 0.05391784837922221    
there was an error with those parameters: Window: 20, timesteps: 50,                 

              ent_coef: 0.01405446007531199, gamma: 0.99, learning_rate: 0.05391784837922221
--------------------------------------------------------------------------------     
Window size: 20, entropy: 0.017429507980349247, lear

...

Min. Reward          : -96257.521                                                      
Avg. Reward          :    801.764                                                      
Max. Reward          : 156299.647                                                      
Min. Orders          :     30.000                                                      
Avg. Orders          :     37.200                                                      
Max. Orders          :     45.000                                                      
--------------------------------------------------------------------------------       
Window size: 20, entropy: 0.01034258067224074, learning rate: 0.02318393727636731      
there was an error with those parameters: Window: 20, timesteps: 50,                   

              ent_coef: 0.01034258067224074, gamma: 0.99, learning_rate: 0.02318393727636731
--------------------------------------------------------------------------------       
Window size: 10, entropy: 

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: 64332.601, # of orders: 42.000:   0%|          | 0/10 [00:05<?, ?it/s]
Episode: 0, Avg. Reward: 64332.601, # of orders: 42.000:  10%|#         | 1/10 [00:05<00:48,  5.34s/it]
Episode: 1, Avg. Reward: 51323.897, # of orders: 41.000:  20%|##        | 2/10 [00:10<00:42,  5.34s/it]
Episode: 1, Avg. Reward: 51323.897, # of orders: 41.000:  30%|###       | 3/10 [00:10<00:23,  3.42s/it]
Episode: 2, Avg. Reward: 43920.647, # of orders: 42.333:  40%|####      | 4/10 [00:17<00:20,  3.42s/it]
Episode: 2, Avg. Reward: 43920.647, # of orders: 42.333:  50%|#####     | 5/10 [00:17<00:16,  3.33s/it]
Episode: 3, Avg. Reward: 7250.781, # of orders: 39.500:  50%|#####     | 5/10 [00:24<00:16,  3.33s/it] 
Episode: 3, Avg. Reward: 7250.781, # of orders: 39.500:  60%|######    | 6/10 [00:24<00:17,  4.28s/it]
Episode: 4, Avg. Reward: 35359.187, # of orders: 41.600:  70%|#######   | 7/10 [00:29<00:12,  4.28s/it]
Episode: 4, Avg. Reward: 35359.187

Min. Reward          : -102758.819                                                     
Avg. Reward          :   3651.419                                                      
Max. Reward          : 147792.813                                                      
Min. Orders          :     31.000                                                      
Avg. Orders          :     39.400                                                      
Max. Orders          :     50.000                                                      
--------------------------------------------------------------------------------       
Window size: 10, entropy: 0.0074392166609244656, learning rate: 0.034609896108327154   
 99%|█████████▉| 248/250 [18:46:06<23:18, 699.06s/trial, best loss: -41850.89925182221]

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: 58811.055, # of orders: 9.000:   0%|          | 0/10 [00:05<?, ?it/s]
Episode: 0, Avg. Reward: 58811.055, # of orders: 9.000:  10%|#         | 1/10 [00:05<00:48,  5.40s/it]
Episode: 1, Avg. Reward: 30796.895, # of orders: 9.000:  20%|##        | 2/10 [00:09<00:43,  5.40s/it]
Episode: 1, Avg. Reward: 30796.895, # of orders: 9.000:  30%|###       | 3/10 [00:09<00:21,  3.02s/it]
Episode: 2, Avg. Reward: 18252.759, # of orders: 9.000:  40%|####      | 4/10 [00:13<00:18,  3.02s/it]
Episode: 2, Avg. Reward: 18252.759, # of orders: 9.000:  50%|#####     | 5/10 [00:13<00:12,  2.50s/it]
Episode: 3, Avg. Reward: 107.968, # of orders: 9.750:  50%|#####     | 5/10 [00:17<00:12,  2.50s/it]  
Episode: 3, Avg. Reward: 107.968, # of orders: 9.750:  60%|######    | 6/10 [00:17<00:11,  2.82s/it]
Episode: 4, Avg. Reward: 12523.993, # of orders: 10.400:  70%|#######   | 7/10 [00:20<00:08,  2.82s/it]
Episode: 4, Avg. Reward: 12523.993, # of or

Min. Reward          : -104579.881                                                     
Avg. Reward          :  -5864.415                                                      
Max. Reward          :  62188.093                                                      
Min. Orders          :      6.000                                                      
Avg. Orders          :     11.100                                                      
Max. Orders          :     18.000                                                      
--------------------------------------------------------------------------------       
Window size: 10, entropy: 0.3537819477224367, learning rate: 0.031572444463292305      
there was an error with those parameters: Window: 10, timesteps: 50,                   

              ent_coef: 0.3537819477224367, gamma: 0.99, learning_rate: 0.031572444463292305
100%|██████████| 250/250 [19:30:08<00:00, 280.84s/trial, best loss: -41850.89925182221]
Best parameters: {'ent_coe

In [34]:
results_df = pd.DataFrame()
for result in trials.results:
    new_row = pd.DataFrame(result, index=[0])
    results_df = pd.concat([results_df, new_row], axis=0)
results_df

,loss,status,eval_time,parameters
0,NaN,fail,1.713570e+09,NaN
0,45402.295026,ok,1.713570e+09,NaN
0,NaN,fail,1.713570e+09,NaN
0,NaN,fail,1.713570e+09,NaN
0,NaN,fail,1.713570e+09,NaN
...,...,...,...,...
0,-801.764193,ok,1.713635e+09,NaN
0,NaN,fail,1.713636e+09,NaN
0,-3651.418669,ok,1.713637e+09,NaN
0,5864.414671,ok,1.713639e+09,NaN


### Plot Results

In [35]:
# # create a dataframe of the rewards
# rewards_df = pd.DataFrame({'rewards': rewards})
# # plot the rewards
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=rewards_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()


In [36]:
# # import the models from /models folder
# import os
# import glob
# # get the list of models
# model_list = glob.glob('models_4_17_24/*.pkl')
# # separate the strings of each model name on _ and get the last element of the string if the string of the model doesn't include 'initial' or 'updated'
# model_list_episode_nbr = [model.split('_')[-1] for model in model_list if 'initial' not in model and 'updated' not in model]
# model_list_episode_nbr = [int(model_name.split('.')[0]) for model_name in model_list_episode_nbr]
# max_episode = max(model_list_episode_nbr)
# # test the last set of 10 episodes
# init_episode = ((int(max_episode)/10) - 10)*10
# # print(max_episode, init_episode)
# models = []
# # test the last set of 10 episodes from init_episode to max_episode
# for nbr in range(int(init_episode), int(max_episode)+10, 10):
#     # set up the appropriate time_points for each of the models in the list
#     env_train.time_points = list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(nbr)), :].index)# make this -nbr not +nbr next time
#     obs_train, info_train = env_train.reset(seed=2024)
#     # find the model name that contains the nbr
#     model_name = [model for model in model_list if str(nbr) in model][0]
#     print(model_name)
#     # load the models into a list
#     models.append(PPO.load(model_name, env=env_train))

In [37]:
# sim_testing = gym_mtsim.MtSimulator(
#     unit='USD',
#     balance=200000.,
#     leverage=100.,
#     stop_out_level=0.2,
#     hedge=True,
#     symbols_filename=FOREX_DATA_PATH
# )

# env_testing = MtEnv(
#     original_simulator=sim_testing,
#     trading_symbols=['EURUSD'],
#     window_size = window_size_param,
#     time_points=list(testing_index_slice),
#     hold_threshold=0.1,
#     close_threshold=0.1,
#     fee=lambda symbol: {
#         # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#         'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#         # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#     }[symbol],
#     symbol_max_orders=2,
#     multiprocessing_processes=2
# )

In [38]:
# model_ppo = PPO.load(f'models_4_17_24\model_25K_5.pkl', env=env_train)

# obs_test, info_test = env_testing.reset(seed=2024)
# done_test = False
# while not done_test:
#     action, _states = model_ppo.predict(obs_test)
#     obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#     done_test = terminated_test or truncated_test
#     # total_reward += reward_test
#     if done_test:
#         break
# try:
#     order_len = len(env_testing.render()['orders'])
# except:
#     order_len = 0
# # print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")


In [39]:
# # if model_dict is still a thing
# for timestep in timesteps_models_dict.keys():
#     models_dict = timesteps_models_dict[timestep]

#     for nbr in range(0, 10):
#         msg = f"{'-'*8} Testing Model {nbr} with {timestep} training timesteps {'-'*8}"
#         print(f"""{msg}\n{'-'*len(msg)}""")
#         reward_across_episodes = []
#         rewards_dict = {}
#         model_results_dict = {}
#         for episode in range(0, 10):   
#             total_reward = 0
#             done_test = False
#             model_ppo = models_dict[f'model_{nbr}']

#             obs_test, info_test = env_train.reset(seed=2024)
#             while not done_test:
#                 action, _states = model_ppo.predict(obs_test)
#                 obs_test, reward_test, terminated_test, truncated_test, info_test = env_train.step(action)
#                 done_test = terminated_test or truncated_test
#                 total_reward += reward_test
#                 if done_test:
#                     break
#             reward_across_episodes.append(total_reward)
#             try:
#                 order_len = len(env_train.render()['orders'])
#             except:
#                 order_len = 0
#             print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")
#         print_stats(reward_across_episodes)
#         model_results_dict[f'model_{nbr}_{timestep}'] = reward_across_episodes
# model_results_df = pd.DataFrame(model_results_dict)

In [40]:
# # plot the rewards for each model over episodes
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=model_results_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()

In [41]:
# # if model_dict is still a thing
# for timestep in ['25K']:
#     for nbr in tqdm(range(1, 10)):
#         msg = f"{'-'*8} Testing Model {nbr} with {timestep} training timesteps {'-'*8}"
#         print(f"""{msg}\n{'-'*len(msg)}""")
#         reward_across_episodes = []
#         number_of_orders_across_episodes = []
#         rewards_dict = {}
#         model_results_dict = {}
#         for episode in tqdm(range(0, 10)):   
#             total_reward = 0
#             done_test = False
#             model_ppo = PPO.load(f'models_4_17_24\model_{timestep}_{nbr}.pkl', env=env_train)

#             obs_test, info_test = env_testing.reset(seed=2024)
#             while not done_test:
#                 action, _states = model_ppo.predict(obs_test)
#                 obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#                 done_test = terminated_test or truncated_test
#                 total_reward += reward_test
#                 if done_test:
#                     break
#             reward_across_episodes.append(total_reward)
#             try:
#                 order_len = len(env_testing.render()['orders'])
#             except:
#                 order_len = 0
#             number_of_orders_across_episodes.append(order_len)
#             print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")
#         print_stats(reward_across_episodes, 'Reward')
#         print_stats(number_of_orders_across_episodes, 'Orders')
#         model_results_dict[f'model_{nbr}_{timestep}'] = reward_across_episodes
# model_results_df = pd.DataFrame(model_results_dict)

In [42]:
# # plot the rewards for each model over episodes
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=model_results_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()

In [43]:
# # if the cluster has been restarted
# for nbr, time_points_idx in zip(range(0, 10), range(0, 500, 50)):
#     # model_ppo.learn(total_timesteps=25000, callback=ProgressBarCallback(100))
#     env_train = MtEnv(
#         original_simulator=sim_train,
#         trading_symbols=['EURUSD'],
#         window_size = window_size_param,
#         time_points=list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(time_points_idx)), :].index),
#         hold_threshold=0.5,
#         close_threshold=0.5,
#         fee=lambda symbol: {
#             # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#             'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#             # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#         }[symbol],
#         symbol_max_orders=2,
#         multiprocessing_processes=2
#     )
#     # obs_train, info_train = env_train.reset(seed=2024)
#     total_reward = 0
#     done_test = False
#     model_ppo = PPO.load(f'models\model_{nbr}.pkl', env=env_train)

#     env_testing = MtEnv(
#         original_simulator=sim_testing,
#         trading_symbols=['EURUSD'],
#         window_size = window_size_param,
#         # time_points=list(testing_index_slice),
#         hold_threshold=0.5,
#         close_threshold=0.5,
#         fee=lambda symbol: {
#             # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#             'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#             # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#         }[symbol],
#         symbol_max_orders=2,
#         multiprocessing_processes=2
#     )
#     obs_test, info_test = env_testing.reset(seed=2024)
#     while not done_test:
#         action, _states = model_ppo.predict(obs_test)
#         obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#         done_test = terminated_test or truncated_test
#         total_reward += reward_test
#         if done_test:
#             break
#     state = env_testing.render()

#     print(
#         f"balance: {state['balance']}, equity: {state['equity']}, margin: {state['margin']}\n"
#         f"free_margin: {state['free_margin']}, margin_level: {state['margin_level']}\n"

#     )
#     # print(state['orders'].Profit.sum())
#     if len(state['orders']) > 0:
#         print(state['orders'].Profit.sum())

In [44]:
# max_episode = 120
# for model_nbr in range(0, int(max_episode)+10, 10):
#     print(f'Model: {model_nbr}')
#     over_episodes_rewards = []
#     over_episodes_balance = []
#     obs_training, info_training = env_train.reset(seed=2024)
#     for episode in range(0, 10):
#         obs_test, info_test = env_testing.reset(seed=2024)
        
#         # model_ppo.learn(total_timesteps=25000, callback=ProgressBarCallback(100))

#         total_reward = 0
#         done_test = False
#         env_train.time_points = list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(model_nbr)), :].index)
#         obs_training, info_training = env_train.reset(seed=2024)
#         model_ppo = PPO.load(f'models\model_{model_nbr}.pkl', env=env_train)

#         while not done_test:
#             action, _states = model_ppo.predict(obs_test)
#             obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#             done_test = terminated_test or truncated_test

#             total_reward += reward_test
#             if done_test:
#                 break
#         over_episodes_balance.append(info_test['balance'])
#         over_episodes_rewards.append(total_reward)
#         print(f'Episode: {episode}, Reward: {total_reward:.3f}, Balance: {info_test["balance"]:.3f}')
#     print_stats(over_episodes_rewards)
#     print_stats(over_episodes_balance)

In [45]:
# data = pd.DataFrame(plot_data)

# sns.set_style('whitegrid')
# plt.figure(figsize=(8, 6))

# for key in plot_data:
#     if key == 'x':
#         continue
#     label = plot_settings[key]['label']
#     line = plt.plot('x', key, data=data, linewidth=1, label=label)

# plt.xlabel('episode')
# plt.ylabel('reward')
# plt.title('Random vs. SB3 Agents')
# plt.legend()
# plt.show()